# Contextual Bandits Demo for VacSIM policy 
The implenmentation of this policy is done using a python package used of implmenting Contextual Bandits called [SpaceBandits](https://pypi.org/project/space-bandits/).

In [1]:
!pip install space-bandits

## Implementing Linear Model
All the necessary scientific libraries are imported. We have used LinearBandits model which is simplest model of the package. As mentioned in the documentation itself, this package maps contexts to expected rewards with linear coefficients.

In [2]:
import numpy as np
import pandas as pd
# from google.colab import files
import math
import matplotlib.pyplot as plt
from space_bandits import LinearBandits 
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('Datasets/VacSIM Input.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       7500 non-null   object 
 1   Name of State              7500 non-null   object 
 2   Susceptible                7500 non-null   int64  
 3   Total predicted cases      7500 non-null   int64  
 4   Death Rate (Predicted)     7500 non-null   float64
 5   Recovery Rate (Predicted)  7500 non-null   float64
 6   Population Share           7500 non-null   float64
 7   Action                     3750 non-null   float64
 8   Normalized Action          3750 non-null   float64
 9   Reward                     3750 non-null   float64
dtypes: float64(6), int64(2), object(2)
memory usage: 586.1+ KB


In [6]:
df.columns

Index(['Date', 'Name of State', 'Susceptible', 'Total predicted cases',
       'Death Rate (Predicted)', 'Recovery Rate (Predicted)',
       'Population Share', 'Action', 'Normalized Action', 'Reward'],
      dtype='object')

## Training the model

In [7]:
df.head()

,Date,Name of State,Susceptible,Total predicted cases,Death Rate (Predicted),Recovery Rate (Predicted),Population Share,Action,Normalized Action,Reward
0,01-09-2020,Assam,30885978,251754,1.029974,45.496794,15.975457,16.0,30.769230,3.475579
1,01-09-2020,Delhi,16165867,494104,1.029945,46.228932,8.594458,4.0,7.692308,296080.856000
2,01-09-2020,Jharkhand,32819208,133576,1.029377,45.964095,16.888024,8.0,15.384620,11009.349390
3,01-09-2020,Maharashtra,106380541,4631183,1.030104,43.097455,57.529182,7.0,13.461540,97005.858510
4,01-09-2020,Nagaland,1943497,27048,1.027802,43.345164,1.012879,17.0,32.692310,0.080389


After normalising the actions coming from ACKTR (Sub model - 1) we have in total 0-100 possible actions i.e. 101 actions. Similarly the context that we considered for our Model is :
* Susceptible Population - 'Susceptible'. 
* Total number of infected cases predicted using SEIR -  'Total predicted cases'.
* Death Rate (calculated using total cases of Deaths using SEIR) -  'Death Rate (Predicted)'.
* Percentage of Share of population of a State -  'Population Share'.
* Recovery Rate (calculated using total cases of Recoveries using SEIR) 'Recovery Rate (Predicted)'.

Using the total possible actions and number of contexts considered, we intialized the constructor of LinearBandits class

In [8]:
num_actions = 101 
num_features = 5 
model = LinearBandits(num_actions, num_features)

The input data is split such a manner that The input training data is considered from 1 September to 15 September each including 50 episodes total of 5 states i.e. 50*5 = 250 for a single date. This method was adopted to make results robust.

On the other hand, the input test data is corresponds to one date only (16 September) for demonstration purpose    

In [9]:
df_train = df[0:3750]  
df_test = df[3750:4000] #this data corresponds to one date only (16 September) for demonstration purpose    
# For future dates the data will be from row 4000 onwards, 250 rows for each date   
df_train.shape, df_test.shape

((3750, 10), (250, 10))

For each iteration we of training we considered an array of context, A chosen action and corresponding reward. These values are passed to .update() method to update LinearBandits model's internal state.  

In [10]:
for i in range(len(df_train)):
    context = df_train[['Susceptible', 'Total predicted cases', 'Death Rate (Predicted)', 'Population Share', 'Recovery Rate (Predicted)']].iloc[i].to_numpy()
    action =  int((df_train[['Normalized Action']]).iloc[i].to_numpy())
    reward =  int(df_train[['Reward']].iloc[i].to_numpy())
    model.update(context, action, reward)
print('Done')

Done


## Predicting the actions

In [11]:
df_test.head()

,Date,Name of State,Susceptible,Total predicted cases,Death Rate (Predicted),Recovery Rate (Predicted),Population Share,Action,Normalized Action,Reward
3750,16-09-2020,Assam,30441898,603248,1.030090,45.700939,15.975457,NaN,NaN,NaN
3751,16-09-2020,Delhi,15375295,1132659,1.030054,46.946875,8.594458,NaN,NaN,NaN
3752,16-09-2020,Jharkhand,32589056,316000,1.030063,46.067722,16.888024,NaN,NaN,NaN
3753,16-09-2020,Maharashtra,97478946,11704661,1.030102,44.238590,57.529182,NaN,NaN,NaN
3754,16-09-2020,Nagaland,1888867,69661,1.029270,43.756191,1.012879,NaN,NaN,NaN


After training the model as above, we can use the .action() method to map a given context to the action with the highest expected reward. The values of action are stored in array to be recorded and exported.


In [12]:
column_names = ["Action", "Date", "Name of State"]
df_10 = pd.DataFrame(columns = column_names)
arr_action = []
for i in range(len(df_test)): 
  new_context =  df_test[['Susceptible', 'Total predicted cases', 'Death Rate (Predicted)', 'Population Share','Recovery Rate (Predicted)']].iloc[i].to_numpy()
  arr_action.append(model.action(new_context))
  df_10.loc[i] = [arr_action[i],df_test['Date'].iloc[i],df_test['Name of State'].iloc[i]]
  # print(f"Action {i} = {arr_action[i]}\t  Date : {df_test['Date'].iloc[i]}\t Location : {df_test['Name of State'].iloc[i]} \t ")
pd.set_option("display.max_rows", None, "display.max_columns", None)
df_10=df_10.sort_values(by=['Name of State'])


/home/arshita/anaconda3/envs/py36_ai/lib/python3.6/site-packages/space_bandits/linear.py:167: RuntimeWarning: covariance is not positive-semidefinite.
  multivariates = [np.random.multivariate_normal(mus[j], covs[j]) for j in range(n_rows)]


Given below is the reward matrix. Here rows represents all possible actions and columns represents the iteration.

In [23]:
df_test_context  = df_test[['Susceptible', 'Total predicted cases', 'Death Rate (Predicted)', 'Population Share','Recovery Rate (Predicted)']].to_numpy()
exp = model.expected_values(df_test_context)
df_exp_reward = pd.DataFrame(exp)
df_exp_reward.T.head(n=5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,3.042016e+07,2.495727e+07,1.144724e+07,7.435456e+06,2.066733e+07,3.155892e+06,2.174625e+06,6.893060e+05,6.891329e+05,1.052022e+06,2.840253e+07,1.769689e+07,414847.689633,1.407176e+07,4.878293e+08,51851.175989,4.428075e+06,26261.362751,6192.918311,69263.962229,-3.261006e+06,3176.433411,1.521554e+05,4561.035381,1.564668e+07,3.343361e+06,-353.244392,1.383605e+07,1.884799e+07,1127.302579,1.820545e+07,70.689680,6.336073e+06,1.992037e+07,101.795535,2.032123e+07,5.304361,21.481955,79.410586,1.638635,2.239189e+07,1.702961e+07,7.816525,-0.119779,-4.322808e+05,1.202765,0.294104,2.953080e+07,-3.590454e+05,-1.875992e+07,22.681621,83.138679,532.028386,3.019428e+07,-0.816769,3.021397e+07,0.0,-2.496681e+07,0.0,-1.353712e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.176417,0.0,0.0,0.0,3.009818e+07,190.912784,0.0,3.010532e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.649409e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.030937e+07
1,1.530700e+07,2.802183e+06,6.245173e+06,4.807902e+06,1.497200e+07,2.752979e+06,1.437223e+05,1.857918e+06,4.118473e+05,2.064079e+07,-2.885568e+05,4.080246e+06,204652.043608,5.128454e+06,1.330140e+08,24484.552922,1.139858e+07,9008.574523,5152.996803,-95198.102437,1.722771e+07,3107.935047,6.954978e+05,2225.436399,4.903166e+06,1.007934e+07,1163.998734,5.830509e+06,9.899138e+06,1657.928096,7.894260e+06,3.986577,1.178509e+07,1.473724e+06,44.221391,5.982326e+06,0.000642,-1.080425,164.133194,-5.084425,-7.332025e+05,-1.190553e+07,-4.879343,-5.831023,-3.149915e+05,-1.957210,0.142380,1.524569e+07,1.619013e+06,1.913805e+07,-25.231895,176.523721,271.087854,1.527859e+07,-3.714607,1.540991e+07,0.0,1.557585e+07,0.0,-3.048969e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.569522,0.0,0.0,0.0,1.532074e+07,93.806488,0.0,1.507435e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.058078e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.531268e+07
2,3.257174e+07,3.057667e+07,1.212982e+07,7.327458e+06,2.273639e+07,3.074650e+06,2.612729e+06,3.453887e+05,7.159218e+05,-6.216672e+06,3.399301e+07,2.169121e+07,473571.901374,1.723505e+07,6.019550e+08,59581.365653,3.526321e+06,27246.430135,5335.497797,98936.055094,-8.451708e+06,3378.889396,-3.135369e+04,5376.073193,1.904930e+07,2.429646e+06,-809.894749,1.642650e+07,2.229750e+07,1079.701820,2.139904e+07,83.541434,5.767780e+06,2.457533e+07,118.802701,2.437790e+07,6.451670,27.654829,66.155433,4.136523,2.971961e+07,2.512097e+07,25.886257,2.170270,-1.094860e+05,2.308627,0.317303,3.148202e+07,-8.716524e+05,-3.148020e+07,35.064691,18.855445,568.607465,3.231268e+07,0.440897,3.228541e+07,0.0,-3.780930e+07,0.0,-5.070747e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.269212,0.0,0.0,0.0,3.217368e+07,205.421173,0.0,3.228089e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.676121e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.244546e+07
3,9.730456e+07,-3.240931e+07,3.652987e+07,4.547106e+07,-2.958834e+06,1.258263e+07,7.085590e+05,9.854870e+06,2.750157e+06,2.122747e+08,2.062002e+07,-2.694604e+07,50012.363411,-2.781282e+07,-1.014039e+09,7695.126313,1.921842e+07,146583.621253,66888.630695,-45224.008406,9.859294e+07,1855.639572,1.109231e+06,-5675.188655,-2.504141e+07,1.526902e+07,10005.320045,-1.577028e+07,-2.636625e+07,837.884575,-2.561696e+07,81.640859,5.269000e+06,-2.046238e+07,-77.553229,-3.387262e+07,-1.443680,-64.911932,28.002787,-65.690752,-1.270970e+08,-1.129883e+08,-882.390567,-68.495684,-2.075408e+07,-26.366361,0.869017,9.846334e+07,5.764295e+06,2.776620e+08,-147.754417,2704.234457,1731.671727,9.702074e+07,-41.591371,9.851644e+07,0.0,2.443508e+08,0.0,-3.225287e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.476069,0.0,0.0,0.0,9.778350e+07,580.427872,0.0,9.485555e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.937523e+07,0.0,0.0,

## Calculating mean action and normalised action
As defined in Dataset README.md

In [14]:
print("Assam", 
"Mean Action:",df_10[0:50]['Action'].mean(), 
"Normalised Action:",100*df_10[0:50]['Action'].mean()/(df_10[0:50]['Action'].mean()+df_10[50:100]['Action'].mean()+df_10[100:150]['Action'].mean()+df_10[150:200]['Action'].mean()+df_10[200:250]['Action'].mean()))


Assam Mean Action: 60.64 Normalised Action: 19.66405084635839


In [15]:
print("Delhi", 
"Mean Action:",df_10[50:100]['Action'].mean(),
"Normalised Action:",100*df_10[50:100]['Action'].mean()/(df_10[0:50]['Action'].mean()+df_10[50:100]['Action'].mean()+df_10[100:150]['Action'].mean()+df_10[150:200]['Action'].mean()+df_10[200:250]['Action'].mean()))


Delhi Mean Action: 63.18 Normalised Action: 20.487709968221026


In [19]:

print("Jharkhand", 
"Mean Action:",df_10[100:150]['Action'].mean(),
"Normalised Action:",100*df_10[100:150]['Action'].mean()/(df_10[0:50]['Action'].mean()+df_10[50:100]['Action'].mean()+df_10[100:150]['Action'].mean()+df_10[150:200]['Action'].mean()+df_10[200:250]['Action'].mean()))

Jharkhand Mean Action: 58.18 Normalised Action: 18.866333744081977


In [20]:

print("Maharashtra", 
"Mean Action:",df_10[150:200]['Action'].mean(),
"Normalised Action:",100*df_10[150:200]['Action'].mean()/(df_10[0:50]['Action'].mean()+df_10[50:100]['Action'].mean()+df_10[100:150]['Action'].mean()+df_10[150:200]['Action'].mean()+df_10[200:250]['Action'].mean()))

Maharashtra Mean Action: 66.22 Normalised Action: 21.473506712497567


In [22]:

print("Nagaland", 
"Mean Action:",df_10[200:250]['Action'].mean(),
"Normalised Action:",100*df_10[200:250]['Action'].mean()/(df_10[0:50]['Action'].mean()+df_10[50:100]['Action'].mean()+df_10[100:150]['Action'].mean()+df_10[150:200]['Action'].mean()+df_10[200:250]['Action'].mean()))

Nagaland Mean Action: 60.16 Normalised Action: 19.50839872884104
